In [1]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages')
sys.path.append('/usr/local/lib/python3.5/dist-packages')
sys.path

['/anaconda3/lib/python36.zip',
 '/anaconda3/lib/python3.6',
 '/anaconda3/lib/python3.6/lib-dynload',
 '',
 '/anaconda3/lib/python3.6/site-packages',
 '/anaconda3/lib/python3.6/site-packages/aeosa',
 '/anaconda3/lib/python3.6/site-packages/pymesh2-0.2.1-py3.6-macosx-10.7-x86_64.egg',
 '/anaconda3/lib/python3.6/site-packages/torchvision-0.1.9-py3.6.egg',
 '/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/NickHarmening/.ipython',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/local/lib/python3.5/dist-packages']

In [2]:
import create_dict
import torch
import os
import json
import utils
import numpy as np

# Clean Data

In [ ]:
#folder_name = '/data2'
#create_dict.clean_obj(folder_name)

# Generate Dataset

In [ ]:
#!pip3 freeze | grep PyWavefront

In [ ]:
#import pywavefront
#mesh2 = pywavefront.Wavefront('/home/.schramm/Documents/3d_retrieval/data/02747177/8bdea15ae5d929d0a2eb129d649f68cf/models/model_normalized.obj', collect_faces='True')


In [ ]:
input_folder = '/data'
suffix = '_test_gpu2' #suffix will append data_trainxxxx.json
max_elements_per_class = 5
create_dict.create_dictionary(input_folder, max_elements_per_class, suffix)

# Initialize Network

In [3]:
import SiameseNet

device = torch.device("cuda:0" if torch.cuda.torch.cuda.is_available() else "cpu")

batch_size = 2
net = SiameseNet.SiameseNet(batch_size)
suffix = '_test' # comment in if not coming from generating the dataset
path_to_params = "models/testing.pt" # if file does not exist or is empty it starts from untrained and later saves to the file

# shift to GPU if available

net.to(device)
working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

In [ ]:
device

# Train Network

In [ ]:
#training parameters

writer_suffix = 'testing'
margin = 0.5
num_epochs = 1
print_batch = 1
lr = 0.0001



In [ ]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir)

# Validation

In [4]:
margin = 0.5
writer_suffix = 'Val'
SiameseNet.val(net, margin, data_dir_val, writer_suffix, working_dir, class_dir, images=True)

8 val triplets generated
Loaded 5032 word vectors.
Number of validation triplets: 8
Doing Evaluation
0 [4 1 0 3 5]
1 [4 1 0 3 5]
2 [4 1 0 3 5]
3 [4 1 0 3 5]
4 [4 1 0 3 5]
5 [4 1 0 3 5]
6 [4 1 0 3 5]
7 [4 1 0 3 5]
precision: 0.625
NDCG: 0.36855738985992403


# Retrieval

In [ ]:
y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir, print_nn=False)

In [ ]:
num_KNN = 4
max_show = 3
utils.retrieve_images(y_pred, ids, data_dir_val, class_dir, num_KNN, max_show, shuffle=True)

In [ ]:
#sentence = 'This is a black chair'
sentence = input("Enter sentence")
num_KNN = 4
utils.retrieve_one_sentence(net, data_dir_val, working_dir, sentence, class_dir, num_KNN)


In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from skimage import io, transform
name = str('images/' + '02747177' + '/' + '1b7d468a27208ee3dad910e221d16b18' + '/models/model_normalized.png')
img = Image.open(name)
#img = img.convert("RGB")
trans = transforms.ToTensor()
tr = trans(img).unsqueeze(0)

In [ ]:
img.load() # required for png.split()
background = Image.new("RGB", img.size, (255, 255, 255))
background.paste(img, mask=img.split()[3]) # 3 is the alpha channel